In [1]:
import os
os.chdir('..')
os.getcwd()

'/home/abraham-pc/Documents/paid_gigs/malaria_prediction_cv'

In [2]:
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, Dense, Multiply, Reshape, Attention, Flatten, Dropout, Add, Permute
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.applications import VGG16, ResNet50, VGG19, EfficientNetB7
import tensorflow_addons as tfa
from keras_self_attention import SeqSelfAttention 
import attention as attn
import mlflow
import random
# Set seeds to make the experiment more reproducible.
seed = 0
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


2024-01-14 13:20:23.673319: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-14 13:20:23.697388: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 13:20:23.697417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 13:20:23.698134: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 13:20:23.702674: I tensorflow/core/platform/cpu_feature_guar

In [4]:
from src.features.build_features import X_train, y_train, X_test, y_test, X_val, y_val, transform_data

In [5]:
def configure_gpu():

    try:
        gpus = tf.config.list_physical_devices('GPU')
        print(f"Found {len(gpus)} GPU(s): {gpus}")
    except Exception as e:
        print("Error configuring GPU: \n", e)
        print("Training will run on CPU")

    if gpus: # type: ignore
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("Memory growth enabled for all Physical GPUs")
        except Exception as e:
            print("Error configuring memory growth: \n", e)

configure_gpu()

Found 1 GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memory growth enabled for all Physical GPUs


2024-01-14 13:20:31.278792: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 13:20:31.302424: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 13:20:31.302574: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
# Define the model


def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    # Load pre-trained VGG16 model without top (fully connected) layers
    base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Get output tensor of the last VGG16 layer
    x = base_model.output

    conv1 = Conv2D(filters = 64, dilation_rate = dilation_rate, kernel_size = (3, 3), activation = 'relu', padding='same')(x)
    maxpool1 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv1)

    # attn1 = Attention(maxpool1, ))

    conv2 = Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool1)
    maxpool2 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv2)

    conv3 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool2)
    maxpool3 = MaxPooling2D(pool_size = (1, 1), strides=(2, 2))(conv3)

    attn = Attention(attention_units)([maxpool3, maxpool3])

    flattened = Flatten()(attn)

    fc1 = Dense(units = 512, activation = 'relu')(flattened)
    drop1 = Dropout(rate = 0.5)(fc1)

    final = Dense(units = num_classes,activation='sigmoid')(drop1)

    model = Model(inputs=base_model.input, outputs=final)

    return model

In [7]:
# Parameters
input_shape = (224, 224, 3)
input_transform = 0
num_classes = 4
num_epochs = 100
lr = 0.0001
batch_size = 16
patience = int(np.sqrt(num_epochs))
val_split = 0.2
dilation_rate = (2, 2)
attention_units = 256

In [8]:
if input_transform == 1:
    X_train = transform_data(X_train, modify=True)
    X_val = transform_data(X_val)
    X_test = transform_data(X_test)
else:
    X_train = transform_data(X_train)
    X_val = transform_data(X_val)
    X_test = transform_data(X_test)

In [9]:
# Build the model
model = build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units)


# Compile the model and print a summary
model.compile(optimizer=Adam(lr), loss='categorical_crossentropy', metrics=['accuracy', tfa.metrics.F1Score(average='macro', num_classes=4)])
model.summary()

2024-01-14 13:20:31.486941: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 13:20:31.487125: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-14 13:20:31.487196: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['normalization[0][0]']   

In [10]:
def testdata_evaluation(model, X_test, y_test):
    actual = np.argmax(y_test, axis=1)
    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(actual, predictions)
    f1_weighted = f1_score(actual, predictions, average='weighted')
    return float(accuracy), float(f1_weighted)

# def testdata_evaluation(model, test_generator, y_test):
#     actual = np.argmax(y_test, axis=1)
#     preds = model.predict(test_generator)
#     predictions = np.argmax(preds, axis=1)
#     accuracy = accuracy_score(actual, predictions)
#     f1_weighted = f1_score(actual, predictions, average='weighted')
#     return float(accuracy), float(f1_weighted)
    

In [11]:
# Define the MLflow experiment
experiment_name = "DCA_SA_from_SE_on_EfficientNet_V3"
experiment = mlflow.set_experiment(experiment_name)

# Define paths
checkpoint_dir = './models/checkpoints/'
model_file = f'./models/model_{experiment_name+experiment.experiment_id}.keras'
os.makedirs( checkpoint_dir + experiment_name, exist_ok=True)
checkpoint_file = checkpoint_dir + experiment_name

# define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_f1_score', 
    patience=patience, 
    mode='max',
    restore_best_weights=True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_file, 
    monitor="val_f1_score",
    mode="max", 
    save_best_only=True,
    verbose=0
)

# Train the model
with mlflow.start_run(experiment_id=experiment.experiment_id):
    mlflow.log_params({
        "learning_rate": lr,
        "dilation_rate": dilation_rate,
        "attention_units": attention_units,
        "num_classes": num_classes,
        "input_shape": input_shape,
        "input_transform": input_transform,
        "batch_size": batch_size,
        "patience": patience,
        "num_epochs": num_epochs,
        "val_split": val_split
    })

    model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        batch_size=batch_size,
        epochs=num_epochs,
        callbacks=[
            early_stopping,
            checkpoint
        ]
    )
    model.save(model_file)
    mlflow.tensorflow.autolog()
    accuracy, f1_weighted = testdata_evaluation(model, X_test, y_test)
    # accuracy, f1_weighted = testdata_evaluation(model, test_generator, y_test)
    mlflow.log_metrics({
        "test accuracy": accuracy,
        "test f1_weighted": f1_weighted
    })
    mlflow.log_artifact(model_file)


Epoch 1/100


2024-01-14 13:20:42.835375: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-01-14 13:20:43.578570: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-14 13:20:43.807474: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-14 13:20:43.853335: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-14 13:20:44.759270: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f74f52c6730 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-14 13:20:44.759289: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 30

9/9 [==============================] - ETA: 0s - loss: 1.3899 - accuracy: 0.2569 - f1_score: 0.2045INFO:tensorflow:Assets written to: ./models/checkpoints/DCA_SA_from_SE_on_EfficientNet_V3/assets


INFO:tensorflow:Assets written to: ./models/checkpoints/DCA_SA_from_SE_on_EfficientNet_V3/assets


9/9 [==============================] - 46s 5s/step - loss: 1.3899 - accuracy: 0.2569 - f1_score: 0.2045 - val_loss: 1.3442 - val_accuracy: 0.3333 - val_f1_score: 0.1250
Epoch 2/100
9/9 [==============================] - 2s 207ms/step - loss: 1.4221 - accuracy: 0.2639 - f1_score: 0.1508 - val_loss: 1.3491 - val_accuracy: 0.3333 - val_f1_score: 0.1250
Epoch 3/100
9/9 [==============================] - 2s 207ms/step - loss: 1.3420 - accuracy: 0.3542 - f1_score: 0.2581 - val_loss: 1.3434 - val_accuracy: 0.3333 - val_f1_score: 0.1250
Epoch 4/100
9/9 [==============================] - 2s 209ms/step - loss: 1.3929 - accuracy: 0.2917 - f1_score: 0.2092 - val_loss: 1.3435 - val_accuracy: 0.3333 - val_f1_score: 0.1250
Epoch 5/100
9/9 [==============================] - 2s 207ms/step - loss: 1.3626 - accuracy: 0.3125 - f1_score: 0.2224 - val_loss: 1.3447 - val_accuracy: 0.3333 - val_f1_score: 0.1250
Epoch 6/100
9/9 [==============================] - 2s 208ms/step - loss: 1.3973 - accuracy: 0.3125 

In [12]:
# test_model_file = f'./models/model_DCA_first_try453954341690039783.keras'
test_model_file = f'./models/checkpoints/DCA_first_try'

test_model = tf.keras.models.load_model(checkpoint_file)
# test_model = tf.keras.models.load_model(test_model_file)

In [13]:
# Classification Report for each type of attack
def print_f1_scores(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True)

    for class_label, metric in report.items():
        try:
            f1_score_float = metric['f1-score']
            print(f"Class {class_label}: F1-score = {round(f1_score_float, 3)}")
        except:
            pass

In [14]:
# def print_testdata_evaluation(model, train_generator, test_generator, test_df):
#     mapping = train_generator.class_indices
#     actual = test_df['species'].map(mapping)
#     preds = model.predict(test_generator)
#     predictions = np.argmax(preds, axis=1)
#     accuracy = accuracy_score(actual, predictions)
#     # f1_weighted = f1_score(actual, predictions, average='weighted')
#     print_f1_scores(actual, predictions)
#     print(f"Test accuracy = {accuracy}")

def print_newdata_evaluation(model, X_test, y_test):
    actual = np.argmax(y_test, axis=1)
    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(actual, predictions)
    f1_weighted = f1_score(actual, predictions, average='weighted')
    print_f1_scores(actual, predictions)
    print(f"Test accuracy = {accuracy}")

In [15]:
print_newdata_evaluation(test_model, X_test, y_test)

2/2 [==============================] - 2s 237ms/step
Class 0: F1-score = 0.0
Class 1: F1-score = 0.0
Class 2: F1-score = 0.0
Class 3: F1-score = 0.492
Class macro avg: F1-score = 0.123
Class weighted avg: F1-score = 0.16
Test accuracy = 0.32608695652173914


/home/abraham-pc/Documents/paid_gigs/malaria_prediction_cv/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/abraham-pc/Documents/paid_gigs/malaria_prediction_cv/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/abraham-pc/Documents/paid_gigs/malaria_prediction_cv/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div